## Imports

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn import metrics

import pandas as pd
import numpy as np

import backtrader as bt

## Data laoding from CSV
### Stockdata
* Load data from CSV

In [3]:
stock_list = pd.read_csv('csv/result_table.csv',sep=',')
u_symbol = stock_list['SYMBOL'].unique()
date = pd.Series(stock_list['Date'])
stock_list.drop(['Date'],axis=1,inplace=True)
stock_list['date'] = date.apply(lambda x: np.datetime64(x))
# stock_data = {}

# for symbol in u_symbol
#     stock_data[symbol] = stock_list[stock_list['SYMBOL'] == symbol]

# stock_data[list(stock_data.keys())[0]]
stock_list

,SYMBOL,1Day,1Week,1Month,3Months,6Months,1Year,2Years,Day1Prior,Day2Prior,...,Month1Prior,Month2Prior,Month3Prior,Month4Prior,Month5Prior,Month6Prior,Month7Prior,Year1Prior,Year2Prior,date
0,ALT,-0.006659,-0.087680,NaN,NaN,NaN,NaN,NaN,0.065012,-0.133654,...,-0.249167,0.174707,2.817797,2.412879,2.085616,2.312500,1.815625,-0.272213,0.202937,2024-02-14
1,JOBY,0.008170,0.024510,-0.151961,0.004902,NaN,NaN,NaN,-0.067073,-0.055556,...,-0.159341,-0.202086,-0.420455,-0.216389,0.313305,0.481840,0.614776,0.478261,-0.322259,2023-10-11
2,TLRY,0.004065,-0.024390,-0.223577,-0.195122,NaN,NaN,NaN,-0.039062,-0.160410,...,-0.068182,0.464286,0.556962,-0.027668,0.069565,-0.075188,-0.115108,-0.236025,-0.797864,2023-09-19
3,CVNA,-0.004679,0.000425,0.602722,1.161208,0.680136,NaN,NaN,0.105833,0.232826,...,1.239048,1.620959,2.334752,1.133394,2.349003,3.867495,1.367573,0.117927,-0.914403,2023-06-13
4,UEC,0.007092,0.056738,-0.106383,0.209220,0.914894,NaN,NaN,0.032967,0.032967,...,-0.203390,-0.286076,-0.255937,-0.267532,-0.331754,-0.124224,-0.369128,-0.422131,-0.075410,2023-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,CCRC,-0.002417,-0.016116,-0.033038,-0.140210,-0.103143,-0.157937,-0.629331,0.001614,0.044613,...,0.035893,0.299476,0.453162,0.411832,-0.376068,-0.328100,-0.250151,-0.108477,-0.119233,2018-11-15
444,SRNEQ,0.119565,0.195652,0.369565,0.543478,-0.032609,0.039130,-0.606522,-0.269841,-0.342857,...,-0.480226,-0.406452,0.260274,1.044444,0.862348,1.628571,1.555556,0.150000,-0.065041,2018-03-28
445,MARK,-0.056402,0.292683,0.350610,-0.076220,-0.376524,-0.696646,-0.914634,-0.056115,-0.290811,...,-0.497318,-0.257919,0.935103,1.122977,1.411765,1.385455,1.095847,1.030960,0.623762,2018-02-07
446,IPO.L,-0.044976,-0.031098,-0.065756,-0.294122,-0.108625,-0.241517,-0.507230,-0.003475,-0.003475,...,0.039580,0.010499,0.151426,0.075150,0.004892,0.078351,0.010499,-0.085454,-0.324780,2017-12-18


In [4]:
print("Number of rows before elimination of NaN: {0}".format(len(stock_list)))
stock_list.dropna(inplace=True)
print("Number of rows after elimination of NaN: {0}".format(len(stock_list)))


Number of rows before elimination of NaN: 448
Number of rows after elimination of NaN: 219


## Create Testsplit
* We create a testsplit at a certain date which allows for roughly 70% of the data to be before it for training.
* The other 30% will be after the date, we ensure this way that when we do out of sample testing that the training wasnt
"poisoned" with future data.

In [71]:
train, test = train_test_split(stock_list, test_size=0.3)
non_test_variables = ['SYMBOL','1Day','1Week','1Month','3Months','6Months','1Year','2Years','date']
predict_variables = ['1Day','1Week','1Month','3Months','6Months','1Year','2Years']

x_train = np.array(train.drop(non_test_variables,axis=1))
y_train = {}
for variable in predict_variables:
    y_train[variable] = np.array(train[variable])

x_test = np.array(test.drop(non_test_variables,axis=1))
y_test ={}
for variable in predict_variables:
    y_test[variable] = np.array(test[variable])


<class 'numpy.ndarray'>


## Train

In [72]:
regressors = {}
for variable in predict_variables: 
    # Define classifier
    clf = xgb.XGBRegressor()
    # Train on trainsplit
    clf.fit(x_train,y_train[variable])
    regressors[variable] = clf

## Test

In [65]:
# predict outsample
for variable in predict_variables: 
    y_pred_insample = clf.predict(x_train)
    mse = mean_squared_error(y_pred_insample, y_train)
    print("Mean squared error insample for variable:{0} = {0}".format(variable, mse))

# predict outsample
y_pred_outsample = clf.predict(x_test)
mse = mean_squared_error(y_pred_outsample, y_test)
print("Mean squared error outsample: {0}".format(mse))


Mean squared error insample: 6.296775128632449e-07
Mean squared error outsample: 0.021812408809097128


### Insample Testing

In [31]:

def binary_convertor(value):
    if value >= 0.5:
        return 1
    else:
        return 0
for x_test, y_test in zip(X_train, Y_train):
    y_pred = clf.predict(x_test)
    y_pred_binary = list(map(binary_convertor,y_pred))
    print("Accuracy:",metrics.accuracy_score(y_test,y_pred_binary))
    print("Precision:",metrics.precision_score(y_test, y_pred_binary,pos_label=str(1),average="macro"))
    print("Recall:",metrics.recall_score(y_test, y_pred_binary,pos_label=str(1),average="macro"))


Accuracy: 0.48660714285714285
Precision: 0.4817800357397706
Recall: 0.48529863350222635
Accuracy: 0.7506738544474394
Precision: 0.575177150596145
Recall: 0.5633392424149806
Accuracy: 0.5303244005641748
Precision: 0.5263459077185613
Recall: 0.5256305371538031
Accuracy: 0.5061601642710473
Precision: 0.4979467469800266
Recall: 0.498152820412626
Accuracy: 0.5760607105898585
Precision: 0.5463017356945892
Recall: 0.5387260484911496
Accuracy: 0.5166840458811262
Precision: 0.5088152357815279
Recall: 0.5077533577533577
Accuracy: 0.5587412587412588
Precision: 0.5349641481150811
Recall: 0.5274492234169653
Accuracy: 0.5
Precision: 0.5208169677926159
Recall: 0.5183357165920938
Accuracy: 0.5308207115181401
Precision: 0.5177735515406303
Recall: 0.5159953158076478
Accuracy: 0.5013812154696132
Precision: 0.5024798368406416
Recall: 0.5024504752089775
Accuracy: 0.5248618784530387
Precision: 0.5257885370643103
Recall: 0.5256680409222783
Accuracy: 0.5021299254526091
Precision: 0.49723169310636095
Recall: 0

c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is igno

Accuracy: 0.4262295081967213
Precision: 0.4374184482585567
Recall: 0.44347063978965817
Accuracy: 0.47705496722138174
Precision: 0.47541864744161744
Recall: 0.4774140291699641
Accuracy: 0.5035
Precision: 0.4952388468013468
Recall: 0.4956437633311973
Accuracy: 0.499739989599584
Precision: 0.4993242529827896
Recall: 0.4993769443446882
Accuracy: 0.5179324894514767
Precision: 0.5106035061003245
Recall: 0.5097953619544502
Accuracy: 0.5074309978768577
Precision: 0.5123845865558243
Recall: 0.5114800694804755
Accuracy: 0.4967391304347826
Precision: 0.49328996045413953
Recall: 0.4937825059101655
Accuracy: 0.505464480874317
Precision: 0.5034371989639675
Recall: 0.5032872724839401
Accuracy: 0.4876681614349776
Precision: 0.48278116090589496
Recall: 0.48396367976916854
Accuracy: 0.4958263772954925
Precision: 0.5115855135658914
Recall: 0.5107825675249132
Accuracy: 0.577639751552795
Precision: 0.5554004703682603
Recall: 0.5497096757281044
Accuracy: 0.49226361031518623
Precision: 0.48328225341648157
Re

c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is igno

Precision: 0.4936322249755085
Recall: 0.49364164700660085
Accuracy: 0.49416342412451364
Precision: 0.49286323133282106
Recall: 0.4931398758700286
Accuracy: 0.48793215916503585
Precision: 0.4890333279837372
Recall: 0.48952995454468073
Accuracy: 0.49606299212598426
Precision: 0.48706994301264844
Recall: 0.4877668962176005
Accuracy: 0.489501312335958
Precision: 0.49267517935297567
Recall: 0.49300281542501195
Accuracy: 0.5009810333551341
Precision: 0.4990306607782926
Recall: 0.49908266929543527
Accuracy: 0.511520737327189
Precision: 0.5085698859788796
Recall: 0.5081246291213082
Accuracy: 0.5543859649122806
Precision: 0.5405061395627433
Recall: 0.5426073716265882
Accuracy: 0.49730820995962316
Precision: 0.49221425130521956
Recall: 0.49238942300699606
Accuracy: 0.5199189736664416
Precision: 0.5187248459492031
Recall: 0.518028569864813
Accuracy: 0.49320652173913043
Precision: 0.4951727583688843
Recall: 0.49536052009456266
Accuracy: 0.49592391304347827
Precision: 0.49230043307079097
Recall: 0.

c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is igno

Accuracy: 0.4523052464228935
Precision: 0.45047424676531
Recall: 0.45215852795470624
Accuracy: 0.4768370607028754
Precision: 0.48161161485319515
Recall: 0.48256910069964054
Accuracy: 0.5152
Precision: 0.5153930909326593
Recall: 0.5152
Accuracy: 0.502411575562701
Precision: 0.49730326384390655
Recall: 0.49735147486497716
Accuracy: 0.5272391505078485
Precision: 0.5238138211099927
Recall: 0.5234780823324094
Accuracy: 0.4774034511092851
Precision: 0.47648283965673743
Recall: 0.47718961838658275
Accuracy: 0.487986743993372
Precision: 0.4858840802237029
Recall: 0.48615827591133387
Accuracy: 0.4912863070539419
Precision: 0.48787191381624784
Recall: 0.48818620689655173
Accuracy: 0.5025
Precision: 0.5013954559238087
Recall: 0.5013812039540784
Accuracy: 0.4802671523982999
Precision: 0.49354656269549885
Recall: 0.4939074953467284
Accuracy: 0.49107901444350044
Precision: 0.49229319788399417
Recall: 0.49239058211757913
Accuracy: 0.5498575498575499
Precision: 0.5439195815626021
Recall: 0.54463751744

c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is igno

Accuracy: 0.4613756613756614
Precision: 0.46241471734892786
Recall: 0.46265568537431745
Accuracy: 0.4090909090909091
Precision: 0.4204545454545454
Recall: 0.4282385834109972
Accuracy: 0.5326315789473685
Precision: 0.5301294262662484
Recall: 0.528637235062307
Accuracy: 0.6584317937701396
Precision: 0.5747502918666494
Recall: 0.6202660180770281
Accuracy: 0.551948051948052
Precision: 0.5512820512820513
Recall: 0.549620253164557
Accuracy: 0.4956616052060738
Precision: 0.4956616052060738
Recall: 0.4956334773054482
Accuracy: 0.48986486486486486
Precision: 0.4896581660887357
Recall: 0.48966514459665145
Accuracy: 0.5016949152542373
Precision: 0.5057191731415264
Recall: 0.5057098765432099
Accuracy: 0.49426605504587157
Precision: 0.49890148166010234
Recall: 0.49891646757859576
Accuracy: 0.5092592592592593
Precision: 0.5140954686959529
Recall: 0.5141211123624707
Accuracy: 0.48661233993015135
Precision: 0.4871717697588911
Recall: 0.4871349322891173
Accuracy: 0.49417249417249415
Precision: 0.494294

c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is igno

### Outsample Testing

In [32]:
amount_0_pred = 0
amount_1_pred = 0
amount_0 = 0
amount_1 = 0
def binary_convertor(value):
    if value >= 0.5:
        return 1
    else:
        return 0
for x_test, y_test in zip(X_test, Y_test):
    y_pred = clf.predict(x_test)
    y_pred_binary = list(map(binary_convertor,y_pred))
    amount_0 = amount_0 + len(np.where(y_test == 0)[0])
    amount_1 = amount_1 + len(np.where(y_test == 1)[0])
    amount_0_pred = amount_0_pred + len(np.where(np.array(y_pred_binary) == 0)[0])
    amount_1_pred = amount_1_pred + len(np.where(np.array(y_pred_binary) == 1)[0])
    # if(y_pred.max() > a ):
    #     a = y_pred.max()
    # if(y_pred.min() < b ):      
    # #     b = y_pred.max()
    print("Accuracy:",metrics.accuracy_score(y_test,y_pred_binary))
    print("Precision:",metrics.precision_score(y_test, y_pred_binary,pos_label=str(1),average="macro"))
    print("Recall:",metrics.recall_score(y_test, y_pred_binary,pos_label=str(1),average="macro"))
# print("Total amount of 0's:{0}\nTotal amount of 1's:{1}".format(amount_0,amount_1))
# print("Total amount of predicted 0's:{0}\nTotal amount of predicted 1's:{1}".format(amount_0_pred,amount_1_pred))

Accuracy: 0.4219512195121951
Precision: 0.4077374064645529
Recall: 0.41437571592210765
Accuracy: 0.4220665499124343
Precision: 0.4184546442151005
Recall: 0.4184546442151005
Accuracy: 0.4111310592459605
Precision: 0.4155139289145053
Recall: 0.4201491569390402
Accuracy: 0.4314079422382672
Precision: 0.4323073286672882
Recall: 0.4327444943116585
Accuracy: 0.44565217391304346
Precision: 0.4390663390663391
Recall: 0.4412878787878788
Accuracy: 0.44525547445255476
Precision: 0.44892253430255064
Recall: 0.45240544548503026
Accuracy: 0.43933823529411764
Precision: 0.43225730605922297
Recall: 0.4334366166557948
Accuracy: 0.387523629489603
Precision: 0.3771228470948932
Recall: 0.38294917680744456
Accuracy: 0.4220665499124343
Precision: 0.4184546442151005
Recall: 0.4184546442151005
Accuracy: 0.4092741935483871
Precision: 0.40565640620889243
Recall: 0.41249105225483174
Accuracy: 0.4340770791075051
Precision: 0.425425636696215
Recall: 0.4301551741178796
Accuracy: 0.41030927835051545
Precision: 0.403

c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is igno

Precision: 0.6
Recall: 0.5678453947368421
Accuracy: 0.49264705882352944
Precision: 0.484375
Recall: 0.48875189271036124
Accuracy: 0.5166666666666667
Precision: 0.49546937296121785
Recall: 0.4965034965034965
Accuracy: 0.3287775246772969
Precision: 0.3320509463159148
Recall: 0.33578062804069
Accuracy: 0.5454545454545454
Precision: 0.5517857142857143
Recall: 0.5479497354497355
Accuracy: 0.48514851485148514
Precision: 0.5471830985915492
Recall: 0.5424408783783784
Accuracy: 0.5257731958762887
Precision: 0.5266106442577031
Recall: 0.5242346938775511
Accuracy: 0.3888888888888889
Precision: 0.37962962962962965
Recall: 0.38421573478476
Accuracy: 0.4155844155844156
Precision: 0.375
Recall: 0.40040650406504064
Accuracy: 0.4533333333333333
Precision: 0.5
Recall: 0.5
Accuracy: 0.8333333333333334
Precision: 0.8492063492063492
Recall: 0.8055555555555556
Accuracy: 0.3684210526315789
Precision: 0.38636363636363635
Recall: 0.38095238095238093
Accuracy: 0.36363636363636365
Precision: 0.2222222222222222
R

c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Admin\Documents\BA\notebook\venv\Lib\site-packages\sklearn\metrics\_classification.py:1549: UserWarning: Note that pos_label (set to '1') is igno

## Backtesting with backtradern

### Test Strategy

In [33]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.log_returns = self.datas[0].log_returns
        self.index = self.datas[0].index
        
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        size = self.log_returns.__len__()
    
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        log_return_history = self.log_returns.get(ago=0,size=size)
        index_history = self.index.get(ago=0,size=size)
        series = []
        for index, log in zip(index_history, log_return_history):
            series.append([index,log])
        forecast = clf.predict(series)
        # if the predictor predicts decline (1) we sell
        if(forecast[-1] > 0.5):
            self.order = self.sell()
        else:
            self.order = self.buy()


        
        # # Check if we are in the market
        # if not self.position:
        #     #TODO recomender goes here
        #     log_return_history = self.log_returns.get(ago=0,size=size)
        #     index_history = self.index.get(ago=0,size=size)
        #     series = []
        #     for index, log in zip(index_history, log_return_history):
        #         series.append([index,log])
        #     forecast = regressor.predict(series)
        #     print(forecast)
        #     # previous close less than the previous close
        #     # BUY, BUY, BUY!!! (with default parameters)
        #     self.log('BUY CREATE, %.2f' % self.dataclose[0])
            
        #     # Keep track of the created order to avoid a 2nd order
        #     self.order = self.buy()

        # else:

        #     # Already in the market ... we might sell
        #     if len(self) >= (self.bar_executed + 5):
        #         # SELL, SELL, SELL!!! (with all possible default parameters)
        #         self.log('SELL CREATE, %.2f' % self.dataclose[0])

        #         # Keep track of the created order to avoid a 2nd order
        #         self.order = self.sell()

### Custom Data feed
We make a custom data feed to inculde the log_returns in our data. We need this to predict. 

In [34]:
class CustomFeed(bt.feeds.PandasData):
    lines = ('datetime', 'open', 'high', 'low',
             'close', 'volume', 'log_returns', 'index')
    params = (
        ('datetime', 'date'),
        ('openinterest',None),
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('log_returns', -1),
        ('index', -1)
    )

In [35]:
def exec(strategy, df):
    cerebro = bt.Cerebro()
    # Add a strategy
    cerebro.addstrategy(strategy)    
    data = CustomFeed(dataname=df)
    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    cerebro.broker.setcash(100000.0)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    plot = cerebro.plot()
    for index, figure in enumerate(plot[0]):
        # return figure
        figure.savefig("figure_{}.png".format(index)) 

# ---------------------------------
test_list
exec(TestStrategy, stocks_after_report['SLI'])
# stock_data['ABAT']['date'] = stock_data['ABAT']['date'].apply(lambda x: np.datetime64(x))
# exec(TestStrategy, stock_data['ABAT'])


Starting Portfolio Value: 100000.00
2021-11-18, Close, 8.01
2021-11-19, SELL EXECUTED, 8.60
2021-11-19, Close, 9.28
2021-11-22, SELL EXECUTED, 9.68
2021-11-22, Close, 8.43
2021-11-23, BUY EXECUTED, 8.40
2021-11-23, Close, 8.70
2021-11-24, SELL EXECUTED, 9.68
2021-11-24, Close, 10.63
2021-11-26, BUY EXECUTED, 9.65
2021-11-26, Close, 9.94
2021-11-29, SELL EXECUTED, 10.58
2021-11-29, Close, 10.89
2021-11-30, SELL EXECUTED, 10.68
2021-11-30, Close, 10.43
2021-12-01, SELL EXECUTED, 11.00
2021-12-01, Close, 10.42
2021-12-02, BUY EXECUTED, 10.39
2021-12-02, Close, 10.08
2021-12-03, BUY EXECUTED, 10.19
2021-12-03, Close, 9.42
2021-12-06, BUY EXECUTED, 8.68
2021-12-06, Close, 8.57
2021-12-07, BUY EXECUTED, 9.35
2021-12-07, Close, 9.40
2021-12-08, SELL EXECUTED, 9.60
2021-12-08, Close, 9.35
2021-12-09, SELL EXECUTED, 9.60
2021-12-09, Close, 9.01
2021-12-10, BUY EXECUTED, 9.20
2021-12-10, Close, 9.04
2021-12-13, SELL EXECUTED, 9.00
2021-12-13, Close, 8.50
2021-12-14, SELL EXECUTED, 8.06
2021-12-1

<IPython.core.display.Javascript object>

In [36]:
# stock_data['ABAT']['date'] = stock_data['ABAT']['date'].apply(lambda x: np.datetime64(x))
# print(type(stock_data['ABAT']['date'].iloc[0].to_pydatetime()))
# stock_data.set_index('date', inplace=True)
# print(test_list)
# exec(TestStrategy, stock_data['SLI'])
# print(stock_data['ABAT'].head)
# print(X_test[0])
stocks_after_report['SLI']

,open,high,low,close,volume,symbol,log_returns,date,signal,index
825228,9.25,9.50,6.75,8.01,18834100.0,SLI,-0.208809,2021-11-18,1,0
825229,8.60,9.30,8.20,9.28,4827500.0,SLI,0.147171,2021-11-19,0,1
825230,9.68,9.68,7.87,8.43,4884700.0,SLI,-0.096065,2021-11-22,1,2
825231,8.40,9.09,8.27,8.70,2973900.0,SLI,0.031526,2021-11-23,0,3
825232,9.68,10.74,9.18,10.63,8061600.0,SLI,0.200357,2021-11-24,0,4
...,...,...,...,...,...,...,...,...,...,...
825794,1.22,1.23,1.14,1.19,1250300.0,SLI,-0.016667,2024-02-22,1,566
825795,1.22,1.22,1.12,1.20,1063900.0,SLI,0.008368,2024-02-23,0,567
825796,1.21,1.21,1.12,1.14,1596000.0,SLI,-0.051293,2024-02-26,1,568
825797,1.18,1.25,1.13,1.25,1914900.0,SLI,0.092115,2024-02-27,0,569


In [37]:
# max_abs_scaler = preprocessing.MaxAbsScaler()
# a = []
# model_dict = {}
# for index, row in short_list.iterrows():
#     if row['SYMBOL'] in list(stocks_after_report.keys()):
#         report_release_date =  row['Date']
#         #todo replace occurances of i with symbol
#         i = symbol
#         # read df into np.array
#         X = np.array(stocks_after_report[i].drop(['signal','date'],axis=1))
#         X = max_abs_scaler.fit_transform(X)
#         Y = np.array(stocks_after_report[i]['signal'])

#         # split
#         X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)


#         model_dict[i] = {}
#         model_dict[i]['X Train'] = X_train
#         model_dict[i]['X Test'] = X_test
#         model_dict[i]['Y Train'] = y_train
#         model_dict[i]['Y Test'] = y_test
        
#         model = svm.SVC(kernel='rbf',decision_function_shape='ovo')
#         # model = svm.SVC(kernel='linear')
#         # model = svm.SVC(kernel='linear',decision_function_shape='ovo')
#         # model = svm.SVC(kernel='rbf',decision_function_shape='ovo')
#         # model = svm.SVC(kernel='poly')
#         # model = svm.SVC(kernel='poly',decision_function_shape='ovo')
#         # model = svm.SVC(kernel='sigmoid')
#         # model = svm.SVC(kernel='sigmoid',decision_function_shape='ovo')

#         # fit and test
#         try:
#             model.fit(X_train, y_train)
#             y_pred = model.predict(X_test)
#             model_dict[i]['Y Prediction'] = y_pred
#             print("SVM Model Info for Ticker: "+i)
#             print("Accuracy:",metrics.accuracy_score(model_dict[i]['Y Test'], model_dict[i]['Y Prediction']))
#             model_dict[i]['Accuracy'] = metrics.accuracy_score(model_dict[i]['Y Test'], model_dict[i]['Y Prediction'])
#             print("Precision:",metrics.precision_score(model_dict[i]['Y Test'], model_dict[i]['Y Prediction'],pos_label=str(1),average="macro"))
#             model_dict[i]['Precision'] = metrics.precision_score(model_dict[i]['Y Test'], model_dict[i]['Y Prediction'],pos_label=str(1),average="macro")
#             print("Recall:",metrics.recall_score(model_dict[i]['Y Test'], model_dict[i]['Y Prediction'],pos_label=str(1),average="macro"))
#             model_dict[i]['Recall'] = metrics.recall_score(model_dict[i]['Y Test'], model_dict[i]['Y Prediction'],pos_label=str(1),average="macro")
#             #print("#################### \n")
#         except: 
#             a.append(i)


In [38]:
# for i in stock_data:
#     #todo is this rly smart
#     stock_data[i]['Returns'] = np.log(stock_data[i]['adj_close']/stock_data[i]['adj_close'].shift(1)) 

#     prediction_length = len(model_dict[i]['Y Prediction'])

#     stock_data[i]['SVM Signal'] = 0
#     stock_data[i]['SVM Returns'] = 0
#     stock_data[i]['Total Strat Returns'] = 0
#     stock_data[i]['Market Returns'] = 0

#     Signal_Column = stock_data[i].columns.get_loc('SVM Signal')
#     Strat_Column = stock_data[i].columns.get_loc('SVM Returns')
#     Return_Column = stock_data[i].columns.get_loc('Total Strat Returns')
#     Market_Column = stock_data[i].columns.get_loc('Market Returns')
        
#     stock_data[i].iloc[-prediction_length:,Signal_Column] = list(map(int,model_dict[i]['Y Prediction']))
#     stock_data[i]['SVM Returns'] = stock_data[i]['SVM Signal'] * stock_data[i]['Returns'].shift(1)

#     stock_data[i].iloc[-prediction_length:,Return_Column] = np.nancumsum(stock_data[i]['SVM Returns'][-prediction_length:])
#     stock_data[i].iloc[-prediction_length:,Market_Column] = np.nancumsum(stock_data[i]['Returns'][-prediction_length:])
#     # print(stock_data[i]['Total Strat Returns'].head)

#     # model_dict[i]['Sharpe_Ratio'] = (stock_data[i]['Total Strat Returns'][-1] - stock_data[i]['Market Returns'][-1])/ \
#     #                 np.nanstd(stock_data[i]['Total Strat Returns'][-prediction_length:])